## 2 - Augmentation des données (Kilométrage)

**Sources :**
- à compléter
- Base de données initial

**Date export :**  jeudi 19 septembre 2024

## 0 - Import des modules et des données

In [323]:
import pandas as pd
import numpy as np

### 0.1 - Dataset 1

In [324]:
filtred_tarifs = pd.read_csv(
    "datasets/tarifs-filtre.csv",
    usecols=[
        "Gare origine", "Gare origine - code UIC",
        "Destination", "Gare destination - code UIC",
        "Prix minimum", "Prix maximum"]
)

In [325]:
filtred_tarifs.head()

,Gare origine,Gare origine - code UIC,Destination,Gare destination - code UIC,Prix minimum,Prix maximum
0,STRASBOURG,87212027,MARNE LA VALLEE CHESSY,87111849,13.0,13.0
1,LAVAL,87478404,MARNE LA VALLEE CHESSY,87111849,9.0,9.0
2,CHAMPAGNE-ARDENNE,87171926,MARNE LA VALLEE CHESSY,87111849,5.0,5.0
3,MARNE LA VALLEE CHESSY,87111849,BOURG ST MAURICE,87741793,14.0,14.0
4,BEZIERS,87781005,MARNE LA VALLEE CHESSY,87111849,15.0,15.0


### 0.2 - Dataset 2

In [326]:
emissions_df = pd.read_csv(
    "datasets/emission-co2-tgv.csv",
    usecols=[
        "Origine", "Origine_uic", 
        "Destination ", "Destination_uic ", 
        "Distance entre les gares"
    ],
    sep=";")

In [327]:
emissions_df.head()

,Origine,Origine_uic,Destination,Destination_uic,Distance entre les gares
0,Paris Montparnasse,87391003,Rennes,87471003,364
1,Paris Montparnasse,87391003,Angers-St-Laud,87484006,303
2,Paris Montparnasse,87391003,Saint-Pierre-des-Corps,87571240,221
3,Paris Est,87113001,Strasbourg,87212027,451
4,Paris Est,87113001,Metz,87192039,352


## 1 - Préparation au merge/concat ;) 

### 1.1 - Vérification de la cohérence des données. 
- Les gares de départ non-redondant -> `UNIQUE`
- Les gares d'arrivée au total -> `COUNT`

In [328]:
print("emissions_df -> unique")
print(f"Destination : {emissions_df["Destination_uic "].nunique()}")
print(f"Origine : {emissions_df["Origine_uic"].nunique()}")

emissions_df -> unique
Destination : 81
Origine : 32


In [329]:
print("emissions_df -> count")
print(f"Destination : {emissions_df["Destination_uic "].count()}")
print(f"Origine : {emissions_df["Origine_uic"].count()}")

emissions_df -> count
Destination : 119
Origine : 119


In [330]:
print("filtred_tarifs")
print(f"Destination : {filtred_tarifs["Gare destination - code UIC"].nunique()}")
print(f"Origine : {filtred_tarifs["Gare origine - code UIC"].nunique()}")

filtred_tarifs
Destination : 169
Origine : 164


In [331]:
print("filtred_tarifs -> count")
print(f"Destination : {filtred_tarifs["Gare destination - code UIC"].count()}")
print(f"Origine : {filtred_tarifs["Gare origine - code UIC"].count()}")

filtred_tarifs -> count
Destination : 1801
Origine : 1801


**Conclusion : Origines et Destinations unique mais une gare de départ peut avoir plusieurs destinations. Il y a un trajet aller et il y a un trajet retours.**

### 1.2 - Compléter le kilométrage déjà présent

In [332]:
filtred_tarifs["Gare destination - code UIC"] = filtred_tarifs['Gare destination - code UIC'].astype("str")
filtred_tarifs["Gare origine - code UIC"] = filtred_tarifs["Gare origine - code UIC"].astype("str")

In [333]:
emissions_df["Destination_uic "] = emissions_df["Destination_uic "].astype("str")
emissions_df["Origine_uic"] = emissions_df["Origine_uic"].astype("str")

In [334]:
filtred_tarifs["concat_uic"] = filtred_tarifs["Gare origine - code UIC"] + filtred_tarifs["Gare destination - code UIC"]
emissions_df["concat_uic"] = emissions_df["Destination_uic "] + emissions_df["Origine_uic"]

In [335]:
filtred_tarifs.head()

,Gare origine,Gare origine - code UIC,Destination,Gare destination - code UIC,Prix minimum,Prix maximum,concat_uic
0,STRASBOURG,87212027,MARNE LA VALLEE CHESSY,87111849,13.0,13.0,8721202787111849
1,LAVAL,87478404,MARNE LA VALLEE CHESSY,87111849,9.0,9.0,8747840487111849
2,CHAMPAGNE-ARDENNE,87171926,MARNE LA VALLEE CHESSY,87111849,5.0,5.0,8717192687111849
3,MARNE LA VALLEE CHESSY,87111849,BOURG ST MAURICE,87741793,14.0,14.0,8711184987741793
4,BEZIERS,87781005,MARNE LA VALLEE CHESSY,87111849,15.0,15.0,8778100587111849


In [336]:
emissions_df.head()

,Origine,Origine_uic,Destination,Destination_uic,Distance entre les gares,concat_uic
0,Paris Montparnasse,87391003,Rennes,87471003,364,8747100387391003
1,Paris Montparnasse,87391003,Angers-St-Laud,87484006,303,8748400687391003
2,Paris Montparnasse,87391003,Saint-Pierre-des-Corps,87571240,221,8757124087391003
3,Paris Est,87113001,Strasbourg,87212027,451,8721202787113001
4,Paris Est,87113001,Metz,87192039,352,8719203987113001


In [337]:
# emission_df = emissions_df[emissions_df["concat_uic"].isin(filtred_tarifs["concat_uic"])]
emissions_df["concat_uic"].isin(filtred_tarifs["concat_uic"]).count()

np.int64(119)

In [338]:
merged_df = filtred_tarifs.merge(
    emissions_df[['concat_uic', 'Distance entre les gares']], 
    on='concat_uic', 
    how='left', 
    indicator=True
    )
filtred_tarifs['Distance entre les gares'] = np.where(
    merged_df['_merge'] == 'both', 
    merged_df['Distance entre les gares'], np.nan
    )

In [339]:
filtred_tarifs.head()

,Gare origine,Gare origine - code UIC,Destination,Gare destination - code UIC,Prix minimum,Prix maximum,concat_uic,Distance entre les gares
0,STRASBOURG,87212027,MARNE LA VALLEE CHESSY,87111849,13.0,13.0,8721202787111849,NaN
1,LAVAL,87478404,MARNE LA VALLEE CHESSY,87111849,9.0,9.0,8747840487111849,NaN
2,CHAMPAGNE-ARDENNE,87171926,MARNE LA VALLEE CHESSY,87111849,5.0,5.0,8717192687111849,NaN
3,MARNE LA VALLEE CHESSY,87111849,BOURG ST MAURICE,87741793,14.0,14.0,8711184987741793,NaN
4,BEZIERS,87781005,MARNE LA VALLEE CHESSY,87111849,15.0,15.0,8778100587111849,NaN


In [340]:
filtred_tarifs[filtred_tarifs["Distance entre les gares"].notnull()].head()

,Gare origine,Gare origine - code UIC,Destination,Gare destination - code UIC,Prix minimum,Prix maximum,concat_uic,Distance entre les gares
6,METZ VILLE,87192039,PARIS EST,87113001,18.0,18.0,8719203987113001,352.0
66,DUNKERQUE,87281006,PARIS NORD,87271007,18.0,18.0,8728100687271007,305.0
68,LYON PART DIEU,87723197,AEROPORT CDG2 TGV ROISSY,87271494,11.0,11.0,8772319787271494,442.0
148,BIARRITZ,87673400,PARIS MONTPARNASSE 1 ET 2,87391003,15.0,15.0,8767340087391003,744.0
212,AVIGNON TGV,87318964,MARNE LA VALLEE CHESSY,87111849,14.0,14.0,8731896487111849,646.0


In [341]:
filtred_tarifs["Distance entre les gares"].isna().sum()

np.int64(1765)

In [342]:
filtred_tarifs["r_concat_uic"] = filtred_tarifs["Gare destination - code UIC"] + filtred_tarifs["Gare origine - code UIC"]
emissions_df["r_concat_uic"] = emissions_df["Origine_uic"] + emissions_df["Destination_uic "]

In [343]:
merged_df = filtred_tarifs.merge(
    emissions_df[['r_concat_uic', 'Distance entre les gares']], 
    on='r_concat_uic', 
    how='left', 
    indicator=True
    )
filtred_tarifs['Distance entre les gares'] = np.where(
    merged_df['_merge'] == 'both', 
    merged_df['Distance entre les gares_y'], 
    filtred_tarifs['Distance entre les gares']
)

filtred_tarifs['Distance entre les gares'] = filtred_tarifs['Distance entre les gares'].interpolate(method='linear', limit_direction='both')

In [344]:
filtred_tarifs['Prix par km'] = np.where(
    filtred_tarifs['Distance entre les gares'].notnull(),
    filtred_tarifs['Prix maximum'] / filtred_tarifs['Distance entre les gares'],
    np.nan
)
filtred_tarifs.head()


,Gare origine,Gare origine - code UIC,Destination,Gare destination - code UIC,Prix minimum,Prix maximum,concat_uic,Distance entre les gares,r_concat_uic,Prix par km
0,STRASBOURG,87212027,MARNE LA VALLEE CHESSY,87111849,13.0,13.0,8721202787111849,352.0,8711184987212027,0.036932
1,LAVAL,87478404,MARNE LA VALLEE CHESSY,87111849,9.0,9.0,8747840487111849,352.0,8711184987478404,0.025568
2,CHAMPAGNE-ARDENNE,87171926,MARNE LA VALLEE CHESSY,87111849,5.0,5.0,8717192687111849,352.0,8711184987171926,0.014205
3,MARNE LA VALLEE CHESSY,87111849,BOURG ST MAURICE,87741793,14.0,14.0,8711184987741793,352.0,8774179387111849,0.039773
4,BEZIERS,87781005,MARNE LA VALLEE CHESSY,87111849,15.0,15.0,8778100587111849,352.0,8711184987781005,0.042614


In [345]:
filtred_tarifs = filtred_tarifs[["Destination", "Gare destination - code UIC", "Prix par km"]]

In [346]:
filtred_tarifs.to_csv("datasets/last_data/prix_par_km.csv")

-- END --